In [ ]:
import json
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

In [ ]:
##Load data 
f = open('../data/events.json', 'r')
d = json.load(f)
d = pd.json_normalize(d['events'])
df = pd.DataFrame(d)

f2 = open('../data/meta.json', 'r')
d2 = json.load(f2)
d2 = pd.json_normalize(d2['meta'])
meta_df = pd.DataFrame(d2)

##Get the list of unique products and sessions
prods = meta_df['productid'].unique().tolist()

sessions = df['sessionid'].unique().tolist()

In [ ]:
## Shuffle data and create training and validation sets (90% of the data is for training)
random.Random(4).shuffle(sessions)
sessions_train = [sessions[i] for i in range(round(0.9 * len(sessions)))]

## Create DataFrames for training and validation sets
train_df = df[df['sessionid'].isin(sessions_train)]

validation_df = df[~df['sessionid'].isin(sessions_train)]


In [ ]:
train_df.head()

In [ ]:
import sqlite3 

conn = sqlite3.connect('data.db')
c = conn.cursor()

In [ ]:
purchase_train = []
# target = []
for i in tqdm(sessions_train):
    temp = train_df[train_df['sessionid'] == i]['productid'].tolist()
    if len(temp) > 1:
        purchase_train.append(temp)
#         target.append(temp[-1])

In [ ]:
train_df[train_df['sessionid'] == 'd2ea7bd3-9235-4a9f-a9ea-d7f296e71318']['productid'].tolist()

In [ ]:
purchase_val = []
val_target = []
for i in tqdm(validation_df['sessionid'].unique()):
    temp = validation_df[validation_df['sessionid'] == i]['productid'].tolist()
    purchase_val.append(temp[:-1])
#     val_target.append(temp[-1])

In [ ]:
len(purchase_val)

In [ ]:
c.execute("DROP TABLE validation")
c.execute("CREATE TABLE validation(prodid str)")
conn.commit()

In [ ]:
valid_data = []
for v in purchase_val:
    for a in v:
        valid_data.append(a)


In [ ]:
train_data = []
for t in purchase_train:
    for b in t:
        train_data.append(b)

In [ ]:
for v in valid_data:
    print(v)
    if v in train_data:
         c.execute("INSERT INTO validation VALUES(?)", (v,))

In [ ]:
conn.commit()

In [48]:
##create the word2vec model
model = Word2Vec(window = 5, sg = 1, hs = 0, alpha=0.03, min_alpha=0.0007, seed = 14, sample=0.001)

In [49]:
vocabs = model.build_vocab(purchase_train, progress_per=200)

In [50]:
##train the model
model.train(purchase_train, total_examples = model.corpus_count, epochs=100, report_delay=1)

(28451094, 33393600)

In [52]:
print(model)

Word2Vec(vocab=6180, vector_size=100, alpha=0.03)


In [51]:
model.save('word2vec_sg.model')

In [43]:
vectors =[]
for vv in train_data:
    try:
        vectors.append(model.wv.get_vector(vv))
    except:
        continue

In [44]:
len(vectors)

325861

In [45]:
len(train_data)

333936

In [39]:
np.save('train_data', train_data)

In [47]:
np.save('sg_wvectors', vectors)